## A simple model for demand and supply of publicly-provided services in a city

In [ ]:
from enum import Enum
import os.path

import numpy as np
import pandas as pd
import geopandas as gpd
import geopy, geopy.distance
import shapely
from sklearn import gaussian_process

from matplotlib import pyplot as plt 
import seaborn as sns
plt.rcParams['figure.figsize']= (20,14)

In [ ]:
## TODO: find way to put this into some global settings
import os
import sys
nb_dir = os.path.dirname(os.getcwd())
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg

In [ ]:
from src.models.city_items import AgeGroup, ServiceArea, ServiceType, SummaryNorm # enum classes for the model

In [ ]:
from src.models.services_supply import ServiceUnit, ServiceEvaluator, UnitFactory, SchoolFactory, LibraryFactory 
from src.models.services_supply import get_random_pos #TODO refactor this fun into common utils
from src.models.process_tools import MappedPositionsFrame, ServiceValues, GridMaker, ValuesPlotter

In [ ]:
quicktest = [ServiceUnit(ServiceType.PoliceStation, 'Duomo', ageDiffusionIn=None), 
        ServiceUnit(ServiceType.PoliceStation, 'Ripamonti', 
                    position=geopy.Point(45.43, 9.201), ageDiffusionIn=None)]
ServiceEvaluator(quicktest).evaluate_services_at(MappedPositionsFrame(get_random_pos(4)))
del quicktest

In [ ]:
## Load scuole
scuoleFile =  '../data/processed/Milano_datiScuole.csv'
schoolLoader = UnitFactory.createLoader(ServiceType.School, scuoleFile)

# Initialise with a default lengthscale of 0.5 km
schoolUnits = schoolLoader.load(meanRadius=0.5)
schoolEval = ServiceEvaluator(schoolUnits)

In [ ]:
## Load scuole
bibliotecheFile =  '../data/processed/Milano_biblioteche.csv'
bibliotecheLoader = UnitFactory.createLoader(ServiceType.Library, bibliotecheFile)

# Initialise with a default lengthscale of 0.5 km
libraryUnits = bibliotecheLoader.load(meanRadius=0.5)

In [ ]:
allEvaluator = ServiceEvaluator(schoolUnits + libraryUnits)

In [ ]:
milanoGridMK = GridMaker({'quartieri':'../data/raw/Milano_specific/Milano_quartieri.geojson'}, gridStep=.5)
milanoGridMK.grid.head()

In [ ]:
valuesGrid = allEvaluator.evaluate_services_at(milanoGridMK.grid)


In [ ]:
## WORK IN PROGRESS - not completed
class ValuesAggregator:
    '''A class to aggregate provided values for mapped positions'''
    
    def aggregate_values(self):
        
        assert self.serviceValues, 'Empty values on grid found, compute first please!'
        
        quartieri = self.subdivisions[self.quartiereKey][self.IDquartiereCol].values
        
        # initialize output
        averagedValues = {service: pd.DataFrame(np.zeros([len(quartieri), len(AgeGroup.all())]), 
                                 index=quartieri, columns=AgeGroup.all()) 
                 for service in self.serviceValues.keys()}
      
        leftOutZones = []
        for quartiere in quartieri:
            bThisQuartiere = (self.IDquartiere == quartiere)
            # loop over service types
            if not bThisQuartiere.any():
                print('Quartiere %s has no points in grid within it' % quartiere)
                leftOutZones.append(quartiere)
                continue
                
            for servType in self.serviceValues.keys():
                for ageGroup, valuesSeries in self.serviceValues[servType].items(): 
                    quartiereValues = [valuesSeries[tuple(p)] for p in self.plotPoints[bThisQuartiere]]
                    averagedValues[servType].loc[quartiere, ageGroup] = np.mean(quartiereValues)
                    
        return averagedValues, leftOutZones

In [ ]:
plotterNew = ValuesPlotter(valuesGrid)
# plots with tricontourf, not so impressive
plotterNew.plot_service_levels(ServiceType.School)

In [ ]:
plotterNew.plot_service_levels(ServiceType.Library)